In [29]:
from cryptography.fernet import Fernet
import pandas as pd

In [4]:
# Step 1: Generate and save the key (run only once to create and save the key)
def generate_key():
    key = Fernet.generate_key()
    print(key)
    with open("./keys/secret.key", "wb") as key_file:
        key_file.write(key)

# Uncomment to generate a new key
# generate_key()

In [30]:
# Step 2: Load the key
def load_key():
    with open("./keys/secret.key", "rb") as key_file:
        return key_file.read()

In [31]:
# Step 3: Encrypt a specific column in the CSV
def encrypt_column(input_csv, output_csv, column_name):
    # Load the key
    key = load_key()
    cipher = Fernet(key)
    
    # Load the CSV file
    df = pd.read_csv(input_csv)
    print("Before", "\n", df.head())
    
    # Encrypt the target column
    df[column_name] = df[column_name].apply(lambda x: cipher.encrypt(str(x).encode()).decode())
    
    # Save the encrypted CSV
    df.to_csv(output_csv, index=False)
    print("-" * 100)
    print("After", "\n", df.head())
    print(f"Encrypted column '{column_name}' and saved to {output_csv}")

In [35]:
# Step 4: Decrypt the column from an encrypted CSV
def decrypt_column(input_csv, output_csv, column_name):
    # Load the key
    key = load_key()
    cipher = Fernet(key)
    
    # Load the encrypted CSV
    df = pd.read_csv(input_csv)
    print("Before", "\n", df.head())
    
    # Decrypt the target column
    df[column_name] = df[column_name].apply(lambda x: cipher.decrypt(x.encode()).decode())
    
    # Save the decrypted CSV
    df.to_csv(output_csv, index=False)
    print("-" * 100)
    print("After", "\n", df.head())
    print(f"Decrypted column '{column_name}' and saved to {output_csv}")

# Usage

#### Encrypt the column

In [36]:
encrypt_column(input_csv="./data/mock-data.csv", output_csv="./output/mock-data-encrypted.csv", column_name="sensitive_column")

Before 
    id   name sensitive_column
0   1  Alice          Secret1
1   2    Bob          Secret2
After 
    id   name                                   sensitive_column
0   1  Alice  gAAAAABnHyNainiNzFPip8bYGbWmtpOeIkFXCJDdLH9Mzg...
1   2    Bob  gAAAAABnHyNa-8S8NVUbAbXaeciTrux9IvrMkZZH_Hpnx6...
Encrypted column 'sensitive_column' and saved to ./output/mock-data-encrypted.csv


#### Decrypt the column

In [37]:
decrypt_column("./output/mock-data-encrypted.csv", "./output/mock-data-decrypted.csv", "sensitive_column")

Before 
    id   name                                   sensitive_column
0   1  Alice  gAAAAABnHyNainiNzFPip8bYGbWmtpOeIkFXCJDdLH9Mzg...
1   2    Bob  gAAAAABnHyNa-8S8NVUbAbXaeciTrux9IvrMkZZH_Hpnx6...
----------------------------------------------------------------------------------------------------
After 
    id   name sensitive_column
0   1  Alice          Secret1
1   2    Bob          Secret2
Decrypted column 'sensitive_column' and saved to ./output/mock-data-decrypted.csv
